Out dataset is compose of 2 classes (squats and deadlifts)
Currently we have extracted around 500+ .npy files for each class.
Separated in 2 folders: squats and deadlifts under the data/keypoints folder.
The data is extracted from the videos using the extract_keypoints.py script.
Our goal in this script is to load the data and create a dataset that can be used for training and testing.
We will combine the data from both classes into a single dataset and create labels for each class.
The labels will be 0 for squats and 1 for deadlifts.

In [ ]:
import sys
import os

# Go up to project root (from inside training/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '../.'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [1]:
import os
import numpy as np

In [2]:
def load_npy(path: str) -> np.ndarray:
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    return np.load(path, allow_pickle=True)  # allow_pickle=True for loading object arrays

In [ ]:
# Let us process every data file in each directory.
# The directory name is the class of the exercise.

# Assign numerical lables to each class
labels = { "squats": 0, "deadlifts": 1, "shoulder_press":2}

temp_samples = []  # List to hold all samples

MAX_FRAMES = 0

for exercise, label in labels.items():
    folder_path = os.path.join(".././data/keypoints", exercise)
    for file in os.listdir(folder_path):
        if file.endswith(".npy"):
            path = os.path.join(folder_path, file)
            sample = np.load(path)
            temp_samples.append((sample, label))
            
            print (f"Loaded {file} from {exercise} with shape {sample.shape}")
            if sample.shape[0] > MAX_FRAMES:
                MAX_FRAMES = sample.shape[0]

# Pass through the sample and pad each sample to the maximum number of frames
# Pass through the samples and pad each sample to the maximum number of frames
X = []  # Feature data
y = []  # Labels

for sample, label in temp_samples:
    # Use the sample directly - it's already a numpy array
    sample_frames = sample.shape[0]
    pad_len = MAX_FRAMES - sample_frames
    
    if pad_len > 0:
        # Make sure we use the same shape for padding
        pad = np.zeros((pad_len, sample.shape[1], sample.shape[2]))  # Preserve all dimensions
        padded_sample = np.concatenate((sample, pad), axis=0)
    else:
        padded_sample = sample
    
    X.append(padded_sample)
    y.append(label)

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

print(f"Processed {len(X)} samples with {MAX_FRAMES} max frames")
print(f"X shape: {X.shape}, y shape: {y.shape}")


Loaded squats_0.npy from squats with shape (85, 33, 4)
Loaded squats_1.npy from squats with shape (85, 33, 4)
Loaded squats_10.npy from squats with shape (71, 33, 4)
Loaded squats_100.npy from squats with shape (97, 33, 4)
Loaded squats_101.npy from squats with shape (97, 33, 4)
Loaded squats_102.npy from squats with shape (195, 33, 4)
Loaded squats_103.npy from squats with shape (195, 33, 4)
Loaded squats_104.npy from squats with shape (236, 33, 4)
Loaded squats_105.npy from squats with shape (236, 33, 4)
Loaded squats_106.npy from squats with shape (109, 33, 4)
Loaded squats_107.npy from squats with shape (109, 33, 4)
Loaded squats_108.npy from squats with shape (112, 33, 4)
Loaded squats_109.npy from squats with shape (112, 33, 4)
Loaded squats_11.npy from squats with shape (71, 33, 4)
Loaded squats_110.npy from squats with shape (123, 33, 4)
Loaded squats_111.npy from squats with shape (123, 33, 4)
Loaded squats_112.npy from squats with shape (96, 33, 4)
Loaded squats_113.npy from 

In [ ]:
final = np.savez_compressed(".././data/keypoints/deadlifts_squats_shoulder_press_2025-06-21.npz", X=X, y=y)

In [ ]:
# test the saved file
loaded = np.load(".././data/keypoints/deadlifts_squats_shoulder_press_2025-06-20_1.npz")

loaded["X"].shape, loaded["y"].shape  # Check the shapes of the loaded data

((1806, 331, 33, 4), (1806,))

In [ ]:
def create_transformer_ready_arrays(data_dir: str = ".././data/keypoints", 
                                  percentile_cutoff: float = 95.0):
    """
    Create numpy arrays ready for transformer training with proper masking
    
    Returns:
        X: Input sequences [batch_size, max_seq_len, num_keypoints, coordinates]
        y: Labels [batch_size]
        attention_masks: Attention masks [batch_size, max_seq_len]
        sequence_lengths: Original sequence lengths [batch_size]
    """
    labels = {"squats": 0, "deadlifts": 1, "shoulder_press": 2}
    temp_samples = []
    sequence_lengths = []
    
    # Load all samples
    for exercise, label in labels.items():
        folder_path = os.path.join(data_dir, exercise)
        if not os.path.exists(folder_path):
            print(f"Warning: {folder_path} does not exist")
            continue
            
        for file in os.listdir(folder_path):
            if file.endswith(".npy"):
                path = os.path.join(folder_path, file)
                try:
                    sample = np.load(path)
                    temp_samples.append((sample, label))
                    sequence_lengths.append(sample.shape[0])
                    print(f"Loaded {file} from {exercise} with shape {sample.shape}")
                except Exception as e:
                    print(f"Error loading {path}: {e}")
    
    if not temp_samples:
        raise ValueError("No samples loaded!")
    
    # Determine max_frames using percentile to avoid extreme outliers
    max_frames = int(np.percentile(sequence_lengths, percentile_cutoff))
    print(f"Using max_frames = {max_frames} ({percentile_cutoff}th percentile)")
    print(f"Sequence length stats - Min: {min(sequence_lengths)}, Max: {max(sequence_lengths)}")
    
    # Filter out sequences that are too long
    filtered_samples = []
    filtered_lengths = []
    for i, (sample, label) in enumerate(temp_samples):
        if sequence_lengths[i] <= max_frames:
            filtered_samples.append((sample, label))
            filtered_lengths.append(sequence_lengths[i])
        else:
            print(f"Filtering out sample with length {sequence_lengths[i]}")
    
    print(f"Kept {len(filtered_samples)} out of {len(temp_samples)} samples")
    
    # Create padded arrays with attention masks
    X = []
    y = []
    attention_masks = []
    final_sequence_lengths = []
    
    for sample, label in filtered_samples:
        seq_len = sample.shape[0]
        
        # Pad sequence
        if seq_len < max_frames:
            pad_len = max_frames - seq_len
            pad = np.zeros((pad_len, sample.shape[1], sample.shape[2]))
            padded_sample = np.concatenate([sample, pad], axis=0)
        else:
            padded_sample = sample
        
        # Create attention mask
        attention_mask = np.zeros(max_frames)
        attention_mask[:seq_len] = 1
        
        X.append(padded_sample)
        y.append(label)
        attention_masks.append(attention_mask)
        final_sequence_lengths.append(seq_len)
    
    # Convert to numpy arrays
    X = np.array(X)
    y = np.array(y)
    attention_masks = np.array(attention_masks)
    final_sequence_lengths = np.array(final_sequence_lengths)
    
    print(f"Final dataset shape:")
    print(f"X: {X.shape}")
    print(f"y: {y.shape}")
    print(f"attention_masks: {attention_masks.shape}")
    print(f"sequence_lengths: {final_sequence_lengths.shape}")
    
    return X, y, attention_masks, final_sequence_lengths

In [6]:
data_new = create_transformer_ready_arrays()

Loaded squats_0.npy from squats with shape (85, 33, 4)
Loaded squats_1.npy from squats with shape (85, 33, 4)
Loaded squats_10.npy from squats with shape (71, 33, 4)
Loaded squats_100.npy from squats with shape (97, 33, 4)
Loaded squats_101.npy from squats with shape (97, 33, 4)
Loaded squats_102.npy from squats with shape (195, 33, 4)
Loaded squats_103.npy from squats with shape (195, 33, 4)
Loaded squats_104.npy from squats with shape (236, 33, 4)
Loaded squats_105.npy from squats with shape (236, 33, 4)
Loaded squats_106.npy from squats with shape (109, 33, 4)
Loaded squats_107.npy from squats with shape (109, 33, 4)
Loaded squats_108.npy from squats with shape (112, 33, 4)
Loaded squats_109.npy from squats with shape (112, 33, 4)
Loaded squats_11.npy from squats with shape (71, 33, 4)
Loaded squats_110.npy from squats with shape (123, 33, 4)
Loaded squats_111.npy from squats with shape (123, 33, 4)
Loaded squats_112.npy from squats with shape (96, 33, 4)
Loaded squats_113.npy from 